In [1]:
import ctypes
import numpy as np
import math
import time

#Define point structure
class HeliosPoint(ctypes.Structure):
    #_pack_=1
    _fields_ = [('x', ctypes.c_uint16),
                ('y', ctypes.c_uint16),
                ('r', ctypes.c_uint8),
                ('g', ctypes.c_uint8),
                ('b', ctypes.c_uint8),
                ('i', ctypes.c_uint8)]
class Dac:
    def __init__(self):
        #Load and initialize library
        self.HeliosLib = ctypes.cdll.LoadLibrary("./libHeliosDacAPI.so")
        self.num_devices = self.HeliosLib.OpenDevices()
        print("Found ", self.num_devices, "Helios DACs")
        # Define limits
        self.xy_max = int(2**12-1)

dac = Dac()

Found  1 Helios DACs


In [99]:
# Define update function
def dacpos(x, y, r, g, b, lum, dac):
    #Constants
    dac_rate = 65000 # very low, change to higher
    dac_idx = 0
    num_points = 1
    
    # Maps x, y from [-1, 1] to [0, xy_max]
    x_dac = int(float((x + 1)/2)*dac.xy_max)
    y_dac = int(float((y + 1)/2)*dac.xy_max)
    
    # Scale the colors by the luminance
    r_dac = int(lum*r)
    g_dac = int(lum*g)
    b_dac = int(lum*b)
    # Make frame
    frame = HeliosPoint(x_dac, y_dac, r_dac, g_dac, b_dac, 255)
    # Post frame
    status_attempts = 0
    while (status_attempts < 512 and dac.HeliosLib.GetStatus(dac_idx) != 1):
        status_attempts += 1
        print('Too fast!')
    dac.HeliosLib.WriteFrame(dac_idx, dac_rate, 0, ctypes.pointer(frame), num_points)

    
def color_correction(arr_color):
    '''
    Corrects the nonlinearities in the color curve 
    '''
    #arr_color = np.exp(np.log(2)*np.power(arr_color,1)) - 1
    # Scale to the lower cutoff (R, G, B)
    lower_cutoff = np.array([0.25, 0.09, 0.09])
    arr_color = arr_color*(1-lower_cutoff) + lower_cutoff
    
    return arr_color

def position_corection(arr_pos):
    '''
    Corrects x-y directions so that (x=-1,y=-1) is bottom left
    Scales the entire pattern so that it does not clip (amps are weird)
    '''
    # Reverse direction of x and y
    arr_pos *= -1
    
    max_scale = 0.75
    return arr_pos*max_scale
    
def dacpos_batch(arr_pos, arr_col, dac):
    '''
    1) Scales and color corrects 
    2) Converts from unit space to DAC coordinates
    3) Produces a DAC compatible frame and displays it
    '''
    # Force pass-by-value
    arr_pos = arr_pos.copy()
    arr_col = arr_col.copy()
    # Scale the pattern 
    arr_pos = position_corection(arr_pos)
    # Format the position array (-1.0-1.0) -> int(0-4095)
    arr_pos = (((arr_pos+1)/2.0)*dac.xy_max).astype(np.int32)
    
    # Perform color correction
    arr_col = color_correction(arr_col)
    # Format the color array (0-1.0) -> int(0-255)
    arr_col = (arr_col*255).astype(np.int32)
    # Fill a heliospoint arr with these values
    num_points = len(arr_pos)
    #print('making a frame with ', num_points)
    
    frameType = HeliosPoint * num_points
    frame = frameType()
    # Fill the frame
    for idx in range(num_points):
        frame[idx] =     HeliosPoint(int(arr_pos[idx, 0]), 
                                     int(arr_pos[idx, 1]), 
                                     int(arr_col[idx, 0]), 
                                     int(arr_col[idx, 1]),
                                     int(arr_col[idx, 2]),
                                     int(255))
    # Write the values out 
    statusAttempts = 0
    # Make 512 attempts for DAC status to be ready. After that, just give up and try to write the frame anyway
    while(statusAttempts < 512 and dac.HeliosLib.GetStatus(0) != 1):
        statusAttempts += 1
            
    dac.HeliosLib.WriteFrame(0, 55000, 0, ctypes.pointer(frame), num_points)
    

In [19]:
T = 1000
theta = np.linspace(0, 2*np.pi, T)
pos_arr = np.zeros((T, 2))
pos_arr[:, 0] = np.cos(theta)
pos_arr[:, 1] = np.sin(theta)

start = (0.1, 1.0, 0.1)
finish = (1.0, 0.1, 1.0)
smooth = np.linspace(start, finish, T//2)

color_arr = np.concatenate([smooth, smooth[::-1]])

dacpos_batch(pos_arr, color_arr, dac)



making a frame with  1000


In [115]:
# Make a tri-colored circle
# Make a tricolor circle
T = 1000
pos_arr = np.zeros((T, 2))
col_arr = np.zeros((T, 3))

# Set Xs and Ys
theta = np.linspace(0, 2*np.pi, T)
pos_arr[:,0] = np.cos(theta)
pos_arr[:,1] = np.sin(theta)


# Tri color the circle
col_arr[theta < np.pi, 0] = 0
col_arr[theta < np.pi, 1] = 0
col_arr[theta < np.pi, 2] = 0.3
col_arr[theta < (2/3)*np.pi, 0] = 0
col_arr[theta < (2/3)*np.pi, 1] = 0.3
col_arr[theta < (2/3)*np.pi, 2] = 0
col_arr[theta < (1/3)*np.pi, 0] = 0.3
col_arr[theta < (1/3)*np.pi, 1] = 0
col_arr[theta < (1/3)*np.pi, 2] = 0

dacpos_batch(pos_arr, col_arr, dac)
print(pos_arr.dtype)

float64


In [98]:
def func(x):
    x = x + 1

x = 5
func(x)
print(x)

5


In [108]:
scales = np.concatenate([np.linspace(1.0, 0.5, 20), np.linspace(0.5, 1, 20)])

import time
while(True):
    for scale in scales:
        new_pos_arr = scale*pos_arr
        dacpos_batch(new_pos_arr, col_arr, dac)
        #print(scale, new_pos_arr[0:4, 0])


KeyboardInterrupt: 

In [116]:
pos_list = [pos_arr, 0.5*pos_arr, 0.1*pos_arr]
col_list = [col_arr, col_arr, col_arr]
print('a: ', pos_list[0].shape, col_list[0].shape)
pos_list, col_list = stitch_patterns(pos_list, col_list)
print('b: ', pos_list[0].shape, col_list[0].shape)

while(True):
    for pos_arr, col_arr in zip(pos_list, col_list):
        print(pos_arr.shape, col_arr.shape)
        dacpos_batch(pos_arr, col_arr, dac)

a:  (1000, 2) (1000, 3)
(1000, 2) (1000, 3)
(499, 2) (499, 3)
(1000, 2) (1000, 3)
(227, 2) (227, 3)
(1000, 2) (1000, 3)
(3726, 2) (3726, 3)
b:  (2,) (3,)
(2,) (3,)


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [7]:
import cv2
def draw_bitmap(arr_pos, arr_color, dims=(400,400)):
    '''
    Draws a pattern as if we were drawing it on a wall
    '''
    # Copy the arrays so we don't change them 
    arr_pos = arr_pos.copy()
    arr_color = arr_color.copy()
    # Map the positions
    arr_pos[:,0] *= -1
    arr_pos = (((arr_pos+1)/2)*dims).astype(np.int)
    # Map the colors
    arr_color = arr_color[:,::-1]
    T = len(arr_pos)
    img = np.zeros(dims+(3,))
    # Draw points
    for t in range(1, T):
        cv2.line(img, arr_pos[t-1], arr_pos[t], color=arr_color[t-1], thickness=2)
    cv2.imshow('img', img)
    cv2.waitKey(0)
    
    

In [111]:
np.zeros((12, 3)).shape

(12, 3)

In [12]:

def stitch_patterns(arr_pos_list, arr_color_list, angular_density=20):
    #TODO - THIS METHOD NEEDS SOME WORK, CONCENTRIC CIRCLES GET MESSED UP 
    stitched_pos = []
    stitched_color = []
    for idx in range(len(arr_pos_list)):
        stitched_pos.append(arr_pos_list[idx])
        stitched_color.append(arr_color_list[idx])
        # If we are between patterns, interpolate them
        if(idx != len(arr_pos_list) -1):
            dist = np.sqrt(np.sum(np.power(arr_pos_list[idx][:,-1]-arr_pos_list[idx+1][:,0], 2)))
            num_gap_points = int(dist*angular_density)
            pos_gap = np.linspace(arr_pos_list[idx][-1,:], arr_pos_list[idx+1][0,:], num_gap_points, dtype=np.float)
            color_gap = np.zeros((num_gap_points, 3), dtype=np.float)
            stitched_pos.append(pos_gap)
            stitched_color.append(color_gap)
    for arr_pos, arr_color in zip(stitched_pos, stitched_color):     
        print(arr_pos.shape, arr_color.shape)
    
    stitched_pos = np.concatenate(stitched_pos)
    stitched_color = np.concatenate(stitched_color)
    print(stitched_pos.shape, stitched_color.shape)
    return stitched_pos, stitched_color
            
    
def make_circular(arr_pos, arr_color):
    return np.concatenate([arr_pos, arr_pos[::-1, :]]), np.concatenate([arr_color, arr_color[::-1, :]])



In [15]:
# Make three parallel lines
T_line = 100

arr_pos_list = []
arr_color_list = []
for c_idx, y in enumerate(range(-1, 2, 1)):
    print(c_idx, y)
    arr_pos_list.append(np.stack([np.linspace(-1, 1, T_line), y*np.ones(T_line)], axis=1))
    arr_color = np.zeros((T_line, 3))
    arr_color[:, c_idx] = np.linspace(0, 1, T_line)
    arr_color_list.append(arr_color.copy())



arr_pos, arr_color = stitch_patterns(arr_pos_list, arr_color_list)
print(arr_pos.shape, arr_color.shape)

arr_pos, arr_color = make_circular(arr_pos, arr_color)
print(arr_pos.shape, arr_color.shape)

dacpos_batch(arr_pos, arr_color, dac)

0 -1
1 0
2 1
(100, 2) (100, 3)
(231, 2) (231, 3)
(100, 2) (100, 3)
(116, 2) (116, 3)
(100, 2) (100, 3)
(647, 2) (647, 3)
(647, 2) (647, 3)
(1294, 2) (1294, 3)
making a frame with  1294


In [14]:
# Make three concentric circles
T_line = 200
theta_line = np.linspace(0, 2*np.pi, T_line)

arr_pos_list = []
arr_color_list = []
for c_idx, scale in enumerate([0.9, 0.95, 1]):
    arr_pos_list.append(np.stack([np.cos(theta_line), np.sin(theta_line)], axis=1)/scale)
    arr_color = np.zeros((T_line, 3))
    arr_color[:, c_idx] = np.linspace(0, 1, T_line)
    arr_color_list.append(arr_color.copy())



arr_pos, arr_color = stitch_patterns(arr_pos_list, arr_color_list, angular_density=0)

dacpos_batch(arr_pos, arr_color, dac)

(200, 2) (200, 3)
(0, 2) (0, 3)
(200, 2) (200, 3)
(0, 2) (0, 3)
(200, 2) (200, 3)
(600, 2) (600, 3)
making a frame with  600


In [ ]:
draw_bitmap(arr_pos, arr_color)

In [71]:
# make a sine wave
T_line = 1000
freq = 4
theta_line = np.linspace(0, 2*np.pi, T_line)
arr_pos = np.stack([np.linspace(-1, 1, T_line), np.sin(freq*theta_line)], axis=1)
arr_color = np.ones((T, 3))
arr_color[:, 0:2] = 0
arr_pos, arr_color = make_circular(arr_pos, arr_color)
dacpos_batch(arr_pos, arr_color, dac)

making a frame with  2000
